In [62]:
import pandas as pd
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sent_tokenize,word_tokenize

In [2]:
cnbc_oilprice_df = pd.read_csv("../../data/crawler_news_data/oilprice_cnbc_news.csv")
effective_news_date = pd.read_csv("../../data/crude_oil_price/effective_news_date_from_2013_3d_rolling.csv")
effective_news_date.index = pd.DatetimeIndex(effective_news_date.date)

In [3]:
effective_news_date["2018-09":]

,Unnamed: 0,date,latest,open,high,low,quantity,percentage
date,,,,,,,,
2018-09-10,51,2018-09-10,67.54,67.82,68.52,67.33,571.17K,-0.31
2018-09-27,38,2018-09-27,72.12,72.01,72.60,71.71,454.55K,0.77
2018-10-09,30,2018-10-09,74.96,74.21,75.28,74.00,623.54K,0.90
2018-10-16,25,2018-10-16,71.92,71.72,72.29,71.02,535.00K,0.20
2018-10-19,22,2018-10-19,69.12,68.68,69.77,68.56,108.31K,0.68
2018-10-22,21,2018-10-22,69.17,69.41,69.65,68.27,27.42K,0.07
2018-10-30,15,2018-10-30,66.18,66.68,67.26,65.33,680.44K,-1.28
2018-11-09,7,2018-11-09,60.19,60.75,60.79,59.26,821.73K,-0.79
2018-11-12,6,2018-11-12,59.93,60.70,61.28,58.68,806.22K,-0.43


## Sentiment Dictionary

In [95]:
sentiment_df = pd.read_csv("../../data/sentiment_dictionary/LoughranMcDonald_MasterDictionary_2016.csv")
pos_dict = pd.read_csv("../../data/sentiment_dictionary/positive-words.txt",sep="\n")
neg_dict = pd.read_csv("../../data/sentiment_dictionary/negative-words.txt",sep="\n")
negative_list = set([str.lower(word) for word in sentiment_df.loc[sentiment_df.Negative !=0].Word]) | set(neg_dict.word)
positive_list = set([str.lower(word) for word in sentiment_df.loc[sentiment_df.Positive !=0].Word]) | set(pos_dict.word)

In [27]:
clean_cnbc_oilprice_df = cnbc_oilprice_df.loc[cnbc_oilprice_df.content.apply(len)>100]
target_df = clean_cnbc_oilprice_df.loc[clean_cnbc_oilprice_df.date.isin(effective_news_date.date)].reset_index()

In [113]:
from_date = "2018-06"
target_df.index = pd.DatetimeIndex(target_df.date)
target_df = target_df[from_date:]
sid = SentimentIntensityAnalyzer()
for i in range(len(target_df.content)):
    pct_change = effective_news_date.percentage[target_df.date[i]]
    try:
        neg = sum(pd.Series(word_tokenize(target_df.content[i])).isin(negative_list))
        pos = sum(pd.Series(word_tokenize(target_df.content[i])).isin(positive_list))
        summarized_txt = summarize(target_df.content[i],word_count=100)
        scores = sid.polarity_scores(summarized_txt)
        if (effective_news_date.percentage[target_df.date[i]] > 0 and pos>neg) or (effective_news_date.percentage[target_df.date[i]] < 0 and pos<neg):
            print(target_df.date[i])
            print("Summary:\n",summarized_txt)
            print("Summarized Feeling",sid.polarity_scores(summarized_txt))
            print("Dictionary Sentiment positive:",pos," negative:",neg)
    except:
        print("Cannot Summarize")

2018-06-20
Summary:
 While petroleum-producing countries are being buoyed by higher global oil prices, the big consumers are taking a hit — especially India, whose energy appetite is growing, its oil minister said at the Vienna OPEC summit Wednesday.
"We are a primary consumer country, our import is very huge… our appetite is growing, and in future we are (a) major consumer," Dharmendra Pradhan, India's petroleum and natural gas minister, told CNBC's Steve Sedgwick.
But gone are the days when there was one sole source of energy, the minister said, arguing that more "reasonable" oil prices are beneficial for both consumers and producers.
Summarized Feeling {'pos': 0.127, 'neu': 0.845, 'neg': 0.028, 'compound': 0.819}
Dictionary Sentiment positive: 12  negative: 7
2018-06-20
Summary:
 OPEC is not part of the Department of Energy of the United States.”The comment refers to President Trump’s tweets about OPEC artificially pushing prices higher and reports that he personally asked Saudi Ara

2018-06-21
Summary:
 Historically, production typically rises ahead of an OPEC quota hike, and this time looks no different.Saudi Arabia, Kuwait and UAE - the key proponents of the OPEC production cut deal - are all showing significantly higher crude exports so far this month, as spigots are opened once again.
June exports have ramped up - over 500,000 bpd higher than last month.In terms of the red area, it shows flows bound for Asia versus the October 2016 level.
Summarized Feeling {'pos': 0.0, 'neu': 0.899, 'neg': 0.101, 'compound': -0.7906}
Dictionary Sentiment positive: 11  negative: 12
2018-06-21
Summary:
 NEW YORK, June 21 (Reuters) - Stocks around the world fell on Thursday and the U.S. dollar slipped from a peak as investors flocked to bonds after weak economic data and on worries about a U.S.-China trade war.
Index provider MSCI said late Wednesday it would reclassify Argentina and Saudi Arabia as emerging market countries next year, broadening the investor base for both count

 SINGAPORE, June 26 (Reuters) - Singapore refining margins, the benchmark for profitability among oil processors in Asia, fell to their lowest in two years, dragged down by lower gasoline margins as refiners have ramped up output.
The (refining) margins are getting very compressed now." Asia's largest refining centre China increased crude throughput by 9 percent from a year ago during the first five months of 2018 to a record, with the bulk of the increase leading to a boost in output of aviation and motor fuel.
Summarized Feeling {'pos': 0.1, 'neu': 0.794, 'neg': 0.106, 'compound': -0.128}
Dictionary Sentiment positive: 5  negative: 9
2018-06-25
Summary:
 Crude oil can go even higher after it spiked above $70 per barrel Tuesday for the first time since May. The latest jump came after the State Department ordered allies to halt their imports from Iran by November.
This tightens supply heading into the second half of the year and into 2019; given these conditions, the door is now open t

Summary:
 LONDON, July 12 (Reuters) - Oil prices rallied on Thursday, recouping some ground following sharp losses the previous session after Libya said it would resume oil exports.
The rally received an extra boost from the International Energy Agency (IEA), which said the world's oil supply cushion "might be stretched to the limit" due to production losses.
"For WTI (U.S. light crude) there is tightness at Cushing, which will be supportive over July and August," said Virendra Chauhan, oil analyst at Energy Aspects in Singapore.
Summarized Feeling {'pos': 0.099, 'neu': 0.842, 'neg': 0.059, 'compound': 0.4019}
Dictionary Sentiment positive: 11  negative: 28
2018-07-12
Summary:
 OPEC’s rejuvenated bid to tame crude prices could soon exhaust the world’s spare capacity cushion, according to the latest monthly report from the International Energy Agency (IEA).
The IEA’s closely-watched report comes shortly after crude had its biggest one-day drop in two years, amid heightened U.S.-China tr

Cannot Summarize
2018-08-07
Summary:
 Russia said Tuesday it will do "everything necessary" to save the 2015 Iran nuclear deal and protect its shared economic interests with Tehran.
The Iran nuclear deal is an accord between Iran and the U.S., Russia, Britain, Germany, France, China and the European Union.
The Russian foreign ministry website published a statement Tuesday stating that Moscow is "deeply disappointed by U.S. steps to reimpose its national sanctions against Iran," before adding, "We will do everything necessary in the interests of preserving and fully implementing the SVPD (Iran nuclear deal)." 
Summarized Feeling {'pos': 0.125, 'neu': 0.84, 'neg': 0.035, 'compound': 0.7787}
Dictionary Sentiment positive: 5  negative: 4
2018-09-10
Summary:
 Earlier in the session, crude had strengthened as growth of U.S. drilling braked and investors anticipated lower supply once new U.S. sanctions against Iran's crude exports kick in from November.
"A higher oil price scenario is built o

Cannot Summarize
Cannot Summarize
Cannot Summarize
Cannot Summarize
2018-10-30
Summary:
 A company behind a multibillion-dollar project to export liquefied natural gas from Louisiana is delaying its investment decision due to problems lining up Chinese buyers amid the ongoing U.S.-China trade dispute.
Trump has pitched U.S. LNG — natural gas chilled to liquid form — to trade partners from China to Poland.
Australia's LNG Limited on Monday said it will not make a final investment decision this year on its Magnolia LNG terminal near Lake Charles, Louisiana.
"We remain hopeful in our ability to bring a final investment decision for Magnolia LNG to the Board of Directors in the first part of 2019." 
Summarized Feeling {'pos': 0.099, 'neu': 0.833, 'neg': 0.068, 'compound': 0.5106}
Dictionary Sentiment positive: 1  negative: 4
2018-10-30
Summary:
 NEW YORK, Oct 30 (Reuters) - Oil prices fell more than 1 percent in choppy trading on Tuesday on signs of rising supply and concern that global ec

Dictionary Sentiment positive: 2  negative: 33
2018-11-09
Summary:
 (Adds oil benchmark, rig count benchmark and Permian rig count) Nov 9 (Reuters) - U.S. energy firms added oil rigs for a fourth time in the last five, keeping the rig count at its highest in over three years even though crude futures were on track to fall for a fifth week in a row to their lowest level since February.
Drillers added 12 oil rigs in the week to Nov. 9, bringing the total count to 886, the highest level since March 2015, General Electric Co's Baker Hughes energy services firm said in its closely followed report on Friday.
Summarized Feeling {'pos': 0.041, 'neu': 0.854, 'neg': 0.105, 'compound': -0.6808}
Dictionary Sentiment positive: 5  negative: 7
2018-11-09
Summary:
 1984@ NEW YORK, Nov 9 (Reuters) - Oil prices fell about 1 percent on Friday as global supply increased and investors worried that fuel demand could slow, putting U.S. crude on track for the longest stretch of daily declines since 1984.
Benc

Summary:
 SINGAPORE, Nov 13 (Reuters) - Oil prices fell by more than 1 percent on Tuesday, with Brent crude sliding below $70 and WTI below $60 per barrel, after U.S. President Donald Trump put pressure on OPEC not to cut supply to prop up the market.
"Sky-high production in the U.S., coupled with incremental barrels coming from Saudi Arabia and Russia is starting to impact oil market balances.
Saudi Energy Minister Khalid al-Falih said on Monday the Organization of the Petroleum Exporting Countries (OPEC) agreed there was a need to cut oil supply next year by around 1 million bpd from October levels to prevent oversupply.
Summarized Feeling {'pos': 0.082, 'neu': 0.864, 'neg': 0.054, 'compound': 0.0072}
Dictionary Sentiment positive: 6  negative: 20
2018-11-12
Summary:
 SINGAPORE, Nov 13 (Reuters) - Oil prices fell by more than one percent early on Tuesday after U.S. President Donald Trump put pressure on OPEC not to cut supply to prop up the market.
Saudi Energy Minister Khalid al-Fal

In [106]:
s = summarize(target_df.content[1],word_count=100)
ss = sid.polarity_scores(s)

In [109]:
ss['neg']>ss['pos']

False

In [114]:
effective_news_date

,Unnamed: 0,date,latest,open,high,low,quantity,percentage
date,,,,,,,,
2013-11-29,1307,2013-11-29,92.72,92.33,93.90,92.06,143.21K,0.48
2013-12-02,1306,2013-12-02,93.82,92.71,94.08,92.56,234.80K,1.19
2013-12-31,1286,2013-12-31,98.42,99.25,99.39,98.15,113.91K,-0.88
2014-01-01,1285,2014-01-01,98.70,98.61,98.76,98.54,-,0.28
2014-01-20,1272,2014-01-20,93.93,94.31,94.31,93.66,-,-0.46
2014-02-05,1260,2014-02-05,97.38,97.67,98.26,96.80,237.67K,0.20
2014-02-17,1252,2014-02-17,100.58,100.59,100.83,100.23,-,0.28
2014-02-27,1244,2014-02-27,102.40,102.57,103.08,101.75,196.49K,-0.19
2014-03-03,1242,2014-03-03,104.92,103.00,105.22,102.95,295.82K,2.27
